In [1]:
import pickle
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
# Load the preprocessed data
def load_data(file_path='iemocap_data.pkl'):
    with open(file_path, 'rb') as f:
        emotion_data = pickle.load(f)
    
    mfcc_features = [item[0] for item in emotion_data]  # MFCC features
    emotion_labels = [item[2] for item in emotion_data]  # Emotion labels
    
    # Updated emotion mapping with all possible IEMOCAP labels
    emotion_mapping = {'neu': 0, 'happy': 1, 'sad': 2, 'ang': 3, 'exc': 4, 'fru': 5, 'sur': 6}
    
    # Filter out undefined emotions (e.g., 'xxx') and map only valid labels
    mfcc_features_filtered = []
    emotion_labels_filtered = []
    
    for mfcc, emotion in zip(mfcc_features, emotion_labels):
        if emotion in emotion_mapping:  # Only map if the emotion is valid
            mfcc_features_filtered.append(mfcc)
            emotion_labels_filtered.append(emotion_mapping[emotion])
    
    # Split dataset into train and test sets (80% train, 20% test)
    mfcc_train, mfcc_test, emotion_train, emotion_test = train_test_split(
        mfcc_features_filtered, emotion_labels_filtered, test_size=0.2, random_state=42)
    
    return mfcc_train, mfcc_test, emotion_train, emotion_test


In [4]:
class IEMOCAPDataset(torch.utils.data.Dataset):
    def __init__(self, mfcc_features, emotion_labels):
        self.mfcc_features = mfcc_features
        self.emotion_labels = emotion_labels
    
    def __len__(self):
        return len(self.mfcc_features)
    
    def __getitem__(self, idx):
        mfcc_tensor = torch.tensor(self.mfcc_features[idx], dtype=torch.float32)
        label = torch.tensor(self.emotion_labels[idx], dtype=torch.long)
        return mfcc_tensor, label

Model Architecture

In [60]:
import torch
from torch import nn
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Encoder, self).__init__()
        # Backbone network to extract frame-level features from MFCCs
        self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Conv1d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim)
        )

        # Fully connected layers to compress frame-level features
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, x):
        # Print shape for debugging


        # Transpose input to (batch_size, input_dim, time_steps) for Conv1d
        # (batch_size, input_dim, time_steps)

        # Print shape after transpose for debugging

        # Pass through Conv1D layers
        conv_out = self.conv_layers(x)  # (batch_size, hidden_dim, time_steps)

        # Take mean over time axis to aggregate time-wise features
        conv_out = torch.mean(conv_out, dim=2)  # (batch_size, hidden_dim)

        # Pass through fully connected layers
        frame_level_features = self.fc(conv_out)  # (batch_size, 128)
        return frame_level_features




# VAE class shared by all layers
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2 * latent_dim)  # Mean and log-variance
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Sigmoid()  # To reconstruct the input features
        )
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        # Encoder
        mu_logvar = self.encoder(x).view(-1, 2, self.latent_dim)
        mu = mu_logvar[:, 0, :]  # Mean
        logvar = mu_logvar[:, 1, :]  # Log-variance
        
        # Reparameterization trick
        z = self.reparameterize(mu, logvar)
        
        # Decoder
        reconstructed_x = self.decoder(z)
        return reconstructed_x, mu, logvar

# Layer 1: Precursor Speaker Representation
class PrecursorSpeakerLayer(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(PrecursorSpeakerLayer, self).__init__()
        self.vae = VAE(input_dim, latent_dim)
    
    def forward(self, x):
        # Use VAE to extract precursor speaker embedding
        reconstructed_x, mu, logvar = self.vae(x)
        return reconstructed_x, mu, logvar

# Layer 2: Disentangled Content Representation
class DisentangledContentLayer(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(DisentangledContentLayer, self).__init__()
        self.vae = VAE(latent_dim, latent_dim)
        
        # A linear layer to project input_dim (128) to latent_dim (16)
        self.project_to_latent = nn.Linear(input_dim, latent_dim)
    
    def forward(self, x, speaker_mu):
        # Project x to the latent dimension
        projected_x = self.project_to_latent(x)  # (batch_size, latent_dim)
        
        # Subtract speaker_mu from the projected x to disentangle content
        content_input = projected_x - speaker_mu  # Ensure matching dimensions
        
        # Pass through the VAE for content disentanglement
        reconstructed_x, content_mu, content_logvar = self.vae(content_input)
        return reconstructed_x, content_mu, content_logvar


# Layer 3: Final Disentangled Speaker Representation
class FinalSpeakerLayer(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(FinalSpeakerLayer, self).__init__()
        self.vae = VAE(latent_dim, latent_dim)
        
        # A linear layer to project input_dim (128) to latent_dim (16)
        self.project_to_latent = nn.Linear(input_dim, latent_dim)
    
    def forward(self, x, content_mu):
        # Project x to the latent dimension
        projected_x = self.project_to_latent(x)  # (batch_size, latent_dim)
        
        # Subtract content_mu from the projected x to get final speaker embedding
        speaker_input = projected_x - content_mu  # Ensure matching dimensions
        
        reconstructed_x, final_speaker_mu, final_speaker_logvar = self.vae(speaker_input)
        return reconstructed_x, final_speaker_mu, final_speaker_logvar


# Full Temporal Aggregation Module (Layer 1 -> Layer 2 -> Layer 3)
class TemporalAggregation(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(TemporalAggregation, self).__init__()
        self.layer1 = PrecursorSpeakerLayer(input_dim, latent_dim)
        self.layer2 = DisentangledContentLayer(input_dim, latent_dim)
        self.layer3 = FinalSpeakerLayer(input_dim, latent_dim)
    
    def forward(self, x):
        # Layer 1: Precursor Speaker Representation
        precursor_output, speaker_mu, speaker_logvar = self.layer1(x)
        
        # Layer 2: Disentangled Content Representation
        content_output, content_mu, content_logvar = self.layer2(x, speaker_mu)
        
        # Layer 3: Final Disentangled Speaker Representation
        final_speaker_output, final_speaker_mu, final_speaker_logvar = self.layer3(x, content_mu)
        
        return {
            'precursor_output': precursor_output,
            'speaker_mu': speaker_mu,
            'speaker_logvar': speaker_logvar,
            'content_output': content_output,
            'content_mu': content_mu,
            'content_logvar': content_logvar,
            'final_speaker_output': final_speaker_output,
            'final_speaker_mu': final_speaker_mu,
            'final_speaker_logvar': final_speaker_logvar
        }
    

class EmotionRecognitionBranch(nn.Module):
    def __init__(self, input_dim, num_emotions):
        super(EmotionRecognitionBranch, self).__init__()
        # A simple feed-forward network for emotion classification
        self.fc_layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_emotions)  # Output: number of emotion classes
        )
    
    def forward(self, x):
        # Pass the input (e.g., MFCC features) through the emotion classification layers
        emotion_logits = self.fc_layers(x)
        return emotion_logits
    
# Adversarial Discriminator (Emotion Classifier for Adversarial Training)
class EmotionAdversarialDiscriminator(nn.Module):
    def __init__(self, latent_dim, num_emotions):
        super(EmotionAdversarialDiscriminator, self).__init__()
        # A simple discriminator for adversarial loss to ensure emotion-invariance
        self.fc_layers = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_emotions)  # Output: emotion classes
        )
    
    def forward(self, speaker_embedding):
        # Predict emotion from speaker embedding (for adversarial training)
        emotion_logits = self.fc_layers(speaker_embedding)
        return emotion_logits
    
# Decoder Module
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        # Decoder to reconstruct the input features from the latent speaker embedding
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),  # Reconstruct the original feature space (e.g., MFCCs)
            nn.Sigmoid()  # Sigmoid for normalized output between 0 and 1 (if needed)
        )
    
    def forward(self, speaker_embedding):
        # Reconstruct the input feature (e.g., MFCCs) from the latent speaker embedding
        reconstructed_input = self.decoder(speaker_embedding)
        return reconstructed_input


# Final Full Model with Encoder, Temporal Aggregation, Emotion Recognition, and Decoder
class FullModelWithEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim, num_emotions):
        super(FullModelWithEncoder, self).__init__()
        # Encoder to extract frame-level features
        self.encoder = Encoder(input_dim, hidden_dim)
        
        # Temporal Aggregation module (Layer 1, Layer 2, Layer 3)
        self.temporal_aggregation = TemporalAggregation(128, latent_dim)  # Output of encoder is 128-dimensional
        
        # Emotion Recognition Branch
        self.emotion_recognition = EmotionRecognitionBranch(128, num_emotions)  # Input dimension from encoder output
        
        # Adversarial Discriminator for emotion-invariance in speaker embedding
        self.emotion_discriminator = EmotionAdversarialDiscriminator(latent_dim, num_emotions)
        
        # Decoder to reconstruct input features from the final speaker embedding
        self.decoder = Decoder(latent_dim, input_dim)
    
    def forward(self, x):
        # Pass input through Encoder to extract frame-level features
        encoded_features = self.encoder(x)  # (batch_size, 128)
        
        # Pass through Temporal Aggregation (Layers 1 -> 2 -> 3)
        temporal_output = self.temporal_aggregation(encoded_features)
        
        # Extract the final speaker embedding from Layer 3
        final_speaker_embedding = temporal_output['final_speaker_mu']
        
        # Pass encoded features through Emotion Recognition Branch
        emotion_logits = self.emotion_recognition(encoded_features)
        
        # Adversarial: Predict emotion from the speaker embedding (for adversarial loss)
        adversarial_emotion_logits = self.emotion_discriminator(final_speaker_embedding)
        
        # Decoder: Reconstruct the original input features from the final speaker embedding
        reconstructed_input = self.decoder(final_speaker_embedding)
        
        return {
            'temporal_output': temporal_output,  # Contains all outputs from Layers 1, 2, 3
            'emotion_logits': emotion_logits,  # Emotion classification from encoder features
            'adversarial_emotion_logits': adversarial_emotion_logits,  # Adversarial emotion prediction
            'reconstructed_input': reconstructed_input  # Reconstructed input from speaker embedding
        }

In [61]:
def build_model(input_dim=13, latent_dim=16, hidden_dim=64, num_emotions=6):
    model = FullModelWithEncoder(input_dim, latent_dim, hidden_dim, num_emotions)
    return model

In [63]:
from tqdm import tqdm

# Training and validation loop
def train_model(model, train_loader, test_loader, num_epochs=100, lr=0.001, save_path='best_model.pth'):
    # Set up optimizer and loss functions
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    emotion_loss_fn = nn.CrossEntropyLoss()
    reconstruction_loss_fn = nn.MSELoss()

    best_val_accuracy = 0.0  # Track the best validation accuracy

    # Training loop
    for epoch in range(num_epochs):
        # Progress bar for each epoch
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        model.train()
        total_emotion_loss, total_reconstruction_loss, total_adversarial_loss = 0, 0, 0
        all_preds, all_labels = [], []

        # Wrap train_loader with tqdm to show progress
        for mfcc_batch, emotion_batch in tqdm(train_loader, desc="Training"):
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(mfcc_batch)
            emotion_logits = output['emotion_logits']
            adversarial_emotion_logits = output['adversarial_emotion_logits']
            reconstructed_mfcc = output['reconstructed_input']

            # Ensure reconstructed_mfcc has the same shape as mfcc_batch
            reconstructed_mfcc = reconstructed_mfcc.unsqueeze(-1).expand_as(mfcc_batch)

            # Compute losses
            emotion_loss = emotion_loss_fn(emotion_logits, emotion_batch)
            reconstruction_loss = reconstruction_loss_fn(reconstructed_mfcc, mfcc_batch)
            adversarial_loss = -emotion_loss_fn(adversarial_emotion_logits, emotion_batch)

            total_loss = emotion_loss + reconstruction_loss + adversarial_loss

            # Backpropagation and optimizer step
            total_loss.backward()
            optimizer.step()

            # Log losses
            total_emotion_loss += emotion_loss.item()
            total_reconstruction_loss += reconstruction_loss.item()
            total_adversarial_loss += adversarial_loss.item()

            # Store predictions for accuracy calculation
            preds = torch.argmax(emotion_logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(emotion_batch.cpu().numpy())

        # Calculate training accuracy
        train_accuracy = accuracy_score(all_labels, all_preds)

        print(f"Train Emotion Loss: {total_emotion_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation phase
        model.eval()
        val_emotion_loss, val_reconstruction_loss, val_adversarial_loss = 0, 0, 0
        all_val_preds, all_val_labels = [], []

        # Wrap test_loader with tqdm to show progress
        with torch.no_grad():
            for mfcc_batch, emotion_batch in tqdm(test_loader, desc="Validating"):
                output = model(mfcc_batch)
                emotion_logits = output['emotion_logits']
                adversarial_emotion_logits = output['adversarial_emotion_logits']
                reconstructed_mfcc = output['reconstructed_input']

                # Ensure reconstructed_mfcc has the same shape as mfcc_batch
                reconstructed_mfcc = reconstructed_mfcc.unsqueeze(-1).expand_as(mfcc_batch)

                emotion_loss = emotion_loss_fn(emotion_logits, emotion_batch)
                reconstruction_loss = reconstruction_loss_fn(reconstructed_mfcc, mfcc_batch)
                adversarial_loss = -emotion_loss_fn(adversarial_emotion_logits, emotion_batch)

                val_emotion_loss += emotion_loss.item()
                val_reconstruction_loss += reconstruction_loss.item()
                val_adversarial_loss += adversarial_loss.item()

                val_preds = torch.argmax(emotion_logits, dim=1).cpu().numpy()
                all_val_preds.extend(val_preds)
                all_val_labels.extend(emotion_batch.cpu().numpy())

        val_accuracy = accuracy_score(all_val_labels, all_val_preds)

        print(f"Val Emotion Loss: {val_emotion_loss / len(test_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

        # Save the model if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with validation accuracy: {val_accuracy:.4f}")

    print("Training finished!")


In [64]:
import torch
from torch.utils.data import DataLoader

# Custom collate function for padding MFCC features
def pad_collate_fn(batch):
    mfccs = [item[0] for item in batch]
    labels = [item[1] for item in batch]

    # Find the maximum length of MFCC sequences (time steps) in this batch
    max_len = max([mfcc.shape[1] for mfcc in mfccs])

    # Ensure all MFCCs have consistent time steps
    padded_mfccs = [torch.nn.functional.pad(mfcc, (0, max_len - mfcc.shape[1])) for mfcc in mfccs]

    # Stack tensors into a batch
    mfcc_tensor = torch.stack(padded_mfccs)

    # Ensure mfcc_tensor is of shape (batch_size, time_steps, input_dim)
 # Debugging: Check the shape of mfcc_tensor after padding

    # Prepare label tensor
    label_tensor = torch.tensor(labels, dtype=torch.long)

    return mfcc_tensor, label_tensor

mfcc_train, mfcc_test, emotion_train, emotion_test = load_data()
# Define a mapping from old labels to new labels
label_mapping = {0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

emotion_train = [label_mapping[label] for label in emotion_train]
emotion_test = [label_mapping[label] for label in emotion_test]

train_dataset = IEMOCAPDataset(mfcc_train, emotion_train)
test_dataset = IEMOCAPDataset(mfcc_test, emotion_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate_fn)

model = build_model(input_dim=13)  
train_model(model, train_loader, test_loader)


Epoch [1/100]


Training: 100%|██████████| 6/6 [00:00<00:00, 42.71it/s]


Train Emotion Loss: 1.7488, Train Accuracy: 0.2872


Validating: 100%|██████████| 2/2 [00:00<00:00, 140.36it/s]


Val Emotion Loss: 1.6895, Val Accuracy: 0.4468
Model saved with validation accuracy: 0.4468
Epoch [2/100]


...

Val Emotion Loss: 1.6398, Val Accuracy: 0.4468
Epoch [100/100]


Training: 100%|██████████| 6/6 [00:00<00:00, 42.01it/s]


Train Emotion Loss: 1.6928, Train Accuracy: 0.2872


Validating: 100%|██████████| 2/2 [00:00<00:00, 201.55it/s]

Val Emotion Loss: 1.6384, Val Accuracy: 0.4468
Training finished!
